#Pushing dataset

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
    

In [ ]:
!unzip /content/drive/MyDrive/generative_design/CarsDataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: CarsDataset/OutputCode/Code_95499.cs  
  inflating: CarsDataset/OutputCode/Code_955.cs  
  inflating: CarsDataset/OutputCode/Code_9550.cs  
  inflating: CarsDataset/OutputCode/Code_95500.cs  
  inflating: CarsDataset/OutputCode/Code_95501.cs  
  inflating: CarsDataset/OutputCode/Code_95502.cs  
  inflating: CarsDataset/OutputCode/Code_95503.cs  
  inflating: CarsDataset/OutputCode/Code_95504.cs  
  inflating: CarsDataset/OutputCode/Code_95505.cs  
  inflating: CarsDataset/OutputCode/Code_95506.cs  
  inflating: CarsDataset/OutputCode/Code_95507.cs  
  inflating: CarsDataset/OutputCode/Code_95508.cs  
  inflating: CarsDataset/OutputCode/Code_95509.cs  
  inflating: CarsDataset/OutputCode/Code_9551.cs  
  inflating: CarsDataset/OutputCode/Code_95510.cs  
  inflating: CarsDataset/OutputCode/Code_95511.cs  
  inflating: CarsDataset/OutputCode/Code_95512.cs  
  inflating: CarsDataset/OutputCode/Code_95513.cs  


In [ ]:
import os
from datasets import Dataset
from tqdm.notebook import tqdm
from huggingface_hub import login

In [ ]:
login()

In [ ]:
path = "CarsDataset"
code_path = os.path.join(path, "OutputCode")
text_path = os.path.join(path, "InputTextsEn.txt")

dataset_dict = {
    "input":[],
    "output":[],
    "text":[],
}
for i in tqdm(range(len(os.listdir(code_path))), desc="Reading code"):
  local_path = os.path.join(code_path, f"Code_{i}.cs")
  with open(local_path, "r") as f:
    code = f.read()
    dataset_dict["output"].append(code)
with open(text_path, "r") as f:
  texts = f.readlines()
  _ = [dataset_dict["input"].append(text) for text in texts]
for i in range(len(dataset_dict["input"])):
  inp = dataset_dict["input"][i]
  out = dataset_dict["output"][i]
  text = f"<s>[INST] {inp} [/INST] {out} </s>"
  dataset_dict["text"].append(text)

Reading code:   0%|          | 0/257688 [00:00<?, ?it/s]

In [ ]:
import shutil
shutil.rmtree("/content/drive")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = Dataset.from_dict(dataset_dict)

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'text'],
        num_rows: 231919
    })
    test: Dataset({
        features: ['input', 'output', 'text'],
        num_rows: 25769
    })
})

In [ ]:
dataset["train"][0]

{'input': 'Generate a car with the following characteristics: length: 290 cm; width: 150 cm; height: 135 cm; wheel width: 27 cm; wheel radius: 36.6 cm; clearance height: 12 cm; bonnet size : small; bonnet shape : rounded; windscreen shape: rounded; boot shape: rounded;\n',
 'output': 'using System.Collections.Generic;\nusing System;\nusing System.Collections;\nnamespace GenerativeDesign.Cars\n{\n    public class Car : CarBase\n    {\n        public Car()\n        {\n            Length = 290f;\n            Width = 150f;\n            Height = 135f;\n\n            WheelWidth = 27f;\n            WheelRadius = 36.6f;\n            WheelRelativeBiasAlongWidth = 0.1f;\n            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };\n\n            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };\n            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 12f) };\n            WheelBaseTopSurface = new Constant(height: 80f

In [ ]:
dataset.push_to_hub("evgmaslov/gen_cars")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/116 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/116 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/evgmaslov/gen_cars/commit/24f0bcd1a405813c432dcd56eb8f51945dfcaa89', commit_message='Upload dataset', commit_description='', oid='24f0bcd1a405813c432dcd56eb8f51945dfcaa89', pr_url=None, pr_revision=None, pr_num=None)

#Llama SFT

In [1]:
!pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [2]:
!pip install wandb datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00


In [4]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging, GenerationConfig
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer
import torch
import gc
import wandb
from tqdm.notebook import tqdm
import re
import random
gc.collect()
#For PyTorch memory management
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

In [5]:
base_model = "ajibawa-2023/Code-Llama-3-8B"
dataset = "evgmaslov/gen_cars"
new_model = "evgmaslov/CodeLlama-7b-Instruct-fp16-cars"

In [6]:
dataset = load_dataset(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/231919 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25769 [00:00<?, ? examples/s]

In [ ]:
dataset["train"][0]["output"]

'using System.Collections.Generic;\nusing System;\nusing System.Collections;\nnamespace GenerativeDesign.Cars\n{\n    public class Car : CarBase\n    {\n        public Car()\n        {\n            Length = 290f;\n            Width = 150f;\n            Height = 135f;\n\n            WheelWidth = 27f;\n            WheelRadius = 36.6f;\n            WheelRelativeBiasAlongWidth = 0.1f;\n            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };\n\n            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };\n            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 12f) };\n            WheelBaseTopSurface = new Constant(height: 80f);\n            GapBetweenWheelAndBase = 3f;\n\n            BodySegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 0.2f }, new List<float>(){ 0.2f, 1f } };\n            BodySegmentsTopSurfaces = new List<Line>() { new CornerRounded(minHeight: 81f, maxHeight: 94.5f, corne

In [ ]:
def add_prompt(row):
  prompt = """You are an expert in writing code to generate car geometry. You receive a description of a car containing its geometric characteristics. Write code in the format of a C# class definition that describes the car geometry. Fill in the class properties according to the user's request."""
  row["text"] = f"<s><<SYS>>{prompt}<</SYS>>[INST]{row['input']}[/INST]{row['output']}</s>"
  return row

In [ ]:
dataset = dataset.map(add_prompt)

Map:   0%|          | 0/231919 [00:00<?, ? examples/s]

Map:   0%|          | 0/25769 [00:00<?, ? examples/s]

In [ ]:
dataset["train"][0]

{'input': 'Generate a car with the following characteristics: length: 290 cm; width: 150 cm; height: 135 cm; wheel width: 27 cm; wheel radius: 36.6 cm; clearance height: 12 cm; bonnet size : small; bonnet shape : rounded; windscreen shape: rounded; boot shape: rounded;\n',
 'output': 'using System.Collections.Generic;\nusing System;\nusing System.Collections;\nnamespace GenerativeDesign.Cars\n{\n    public class Car : CarBase\n    {\n        public Car()\n        {\n            Length = 290f;\n            Width = 150f;\n            Height = 135f;\n\n            WheelWidth = 27f;\n            WheelRadius = 36.6f;\n            WheelRelativeBiasAlongWidth = 0.1f;\n            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };\n\n            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };\n            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 12f) };\n            WheelBaseTopSurface = new Constant(height: 80f

In [7]:
quant_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_compute_dtype=torch.float16,
                                  bnb_4bit_quant_type="nf4",
                                  bnb_4bit_use_double_quant=True)

In [ ]:
torch.cuda.empty_cache()
gc.collect()

365

In [8]:
model = AutoModelForCausalLM.from_pretrained(base_model,
                                             quantization_config=quant_config,
                                             device_map='auto',
                                  low_cpu_mem_usage=True, offload_state_dict=True)
#model = prepare_model_for_kbit_training(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at ajibawa-2023/Code-Llama-3-8B and are newly initialized: ['model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.29.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [9]:
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/generative_design/checkpoint-1475")

In [ ]:
model = prepare_model_for_kbit_training(model)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_params = LoraConfig(
    lora_alpha=16, lora_dropout=0.1, r=64, bias="none", task_type="CAUSAL_LM")

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

In [ ]:
training_params = TrainingArguments(
    output_dir="/content/drive/MyDrive/generative_design",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25, logging_steps=25,
    learning_rate=3e-5, weight_decay=0.001,
    fp16=False, bf16=False,
    max_grad_norm=0.3,
    max_steps=-1, warmup_ratio=0.001,
    group_by_length=True, lr_scheduler_type='cosine',
    report_to="wandb",
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=25)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"].select(range(100)),
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params, packing=False)
gc.collect()

torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/231919 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
wandb_callback = LLMSampleCB(trainer, dataset["test"], num_samples=2, max_new_tokens=2048)
trainer.add_callback(wandb_callback)

In [ ]:
from peft import PeftModelForCausalLM
help(PeftModelForCausalLM.generate)

Help on function generate in module peft.peft_model:

generate(self, **kwargs)



In [ ]:
wandb.init("generative_design")
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
25,2.029200,2.081352
50,2.115700,1.997082
75,1.841200,1.748803
100,1.600000,1.299104
125,0.948400,0.627938
150,0.423200,0.245163
175,0.176800,0.144462
200,0.121900,0.105992
225,0.094200,0.100627
250,0.098600,0.089313


  0%|          | 0/2 [00:00<?, ?it/s]

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
wandb: Adding directory to artifact (/content/drive/MyDrive/generative_design/checkpoint-1425)... Done. 6.2s
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for mor

  0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
wandb: Adding directory to artifact (/content/drive/MyDrive/generative_design/checkpoint-1450)... Done. 1.4s
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for mor

  0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
wandb: Adding directory to artifact (/content/drive/MyDrive/generative_design/checkpoint-1475)... Done. 4.4s
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for mor

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

In [ ]:
# Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

##Classes

In [ ]:
from transformers import GenerationConfig
from transformers.integrations import WandbCallback
import random
class LLMSampleCB(WandbCallback):
    def __init__(self, trainer, test_dataset, num_samples=2, max_new_tokens=2048, log_model="checkpoint"):
        super().__init__()
        self._log_model = log_model
        self.sample_dataset = test_dataset.select(random.choices(range(len(test_dataset)), k=num_samples))
        self.model, self.tokenizer = trainer.model, trainer.tokenizer
        self.inference_model = None
        self.gen_config = GenerationConfig.from_pretrained(trainer.model.name_or_path,
                                                           max_new_tokens=max_new_tokens)
        self.gen_config.temperature = 0.9
    def generate(self, prompt):
        tokenized_prompt = self.tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
        with torch.inference_mode():
            output = self.inference_model.generate(**{"input_ids":tokenized_prompt,
                                                      "generation_config":self.gen_config})
        return self.tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)

    def samples_table(self, examples):
        records_table = wandb.Table(columns=["prompt", "generation"] + list(self.gen_config.to_dict().keys()))
        for example in tqdm(examples, leave=False):
            system = """You are an expert in writing code to generate car geometry. You receive a description of a car containing its geometric characteristics. Write code in the format of a C# class definition that describes the car geometry. Fill in the class properties according to the user's request."""
            prompt = f"<s><<SYS>>{system}<</SYS>>[INST]{example['input']}[/INST]"
            generation = self.generate(prompt=prompt)
            records_table.add_data(prompt, generation, *list(self.gen_config.to_dict().values()))
        return records_table

    def on_evaluate(self, args, state, control,  **kwargs):
        super().on_evaluate(args, state, control, **kwargs)
        self.inference_model = self.model
        records_table = self.samples_table(self.sample_dataset)
        self._wandb.log({"sample_predictions":records_table})


#Evaluation

In [15]:
num_eval_samples = 100
inds = random.choices(range(len(dataset["test"])), k=num_eval_samples)
eval_samples = dataset["test"].select(inds)

In [ ]:
test_true = ""
test_pred = ""

In [16]:
file_accuracy = 0
pos_count = 0
pbar = tqdm(enumerate(eval_samples), total=num_eval_samples, desc="Evaluation ")
for i, sample in pbar:
  system = """You are an expert in writing code to generate car geometry. You receive a description of a car containing its geometric characteristics. Write code in the format of a C# class definition that describes the car geometry. Fill in the class properties according to the user's request."""
  prompt = f"<s><<SYS>>{system}<</SYS>>[INST]{sample['input']}[/INST]"
  tokenized_prompt = tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
  gen_config = GenerationConfig.from_pretrained(model.name_or_path)
  with torch.inference_mode():
      output = model.generate(**{"input_ids":tokenized_prompt, "generation_config":gen_config})
  output = tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)
  test_pred = output
  test_true = sample["output"]
  output = re.search("using (?:.|\n)*}", output)[0]
  if evaluate(sample["input"], output):
    pos_count += 1
  file_accuracy = pos_count/(i+1)
  pbar.set_description("File accuracy: %s" % file_accuracy)

Evaluation :   0%|          | 0/100 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene

In [14]:
def evaluate(input_text, output_text):
  try:
    length = re.search("(Length = )([^;]+)(;\s+)", output_text).group(2).split("f")[0]
    width = re.search("(Width = )([^;]+)(;\s+)", output_text).group(2).split("f")[0]
    height = re.search("(Height = )([^;]+)(;\s+)", output_text).group(2).split("f")[0]
    wheel_width = re.search("(WheelWidth = )([^;]+)(;\s+)", output_text).group(2).split("f")[0]
    wheel_radius = re.search("(WheelRadius = )([^;]+)(;\s+)", output_text).group(2).split("f")[0]
    clearance = re.search("WheelBaseSegmentsBottomSurfaces = new List<Line>\(\) { new Constant\(height: (.*)f\) };", output_text).group(1)
    body_segment_spans = re.search("BodySegmentsSpans = (.*);", output_text).group(1)
    body_segment_top_surfaces = re.search("BodySegmentsTopSurfaces = (.*);", output_text).group(1)

    real_text = "Generate a car with the following characteristics:"
    real_text = real_text + f" length: {length} cm;"
    real_text = real_text + f" width: {width} cm;"
    real_text = real_text + f" height: {height} cm;"
    real_text = real_text + f" wheel width: {wheel_width} cm;"
    real_text = real_text + f" wheel radius: {wheel_radius} cm;"
    real_text = real_text + f" clearance height: {clearance} cm;"

    bonnet_size = float(re.search("{ new List<float>\(\){ 0f, (.*)f }, new", body_segment_spans).group(1))
    if bonnet_size <= 0.2:
      real_text = real_text + f" bonnet size : small;"
    elif bonnet_size >= 0.3:
      real_text = real_text + f" bonnet size : large;"
    surfaces = body_segment_top_surfaces.split("new")[2:]
    if "CornerRounded" in surfaces[0] and "leftCornerRounded: true" in surfaces[0] or "TotalRounded" in surfaces[0]:
      real_text = real_text + f" bonnet shape : rounded;"
    elif "Constant" in surfaces[0]:
      real_text = real_text + f" bonnet shape : flat;"

    if (len(surfaces) == 2 and "CornerRounded" in surfaces[1] and "leftCornerRounded: true" in surfaces[1]) or (len(surfaces) == 3 and ("TotalRounded" in surfaces[1] or "CornerRounded" in surfaces[1] and "leftCornerRounded: true" in surfaces[1])):
      real_text = real_text + f" windscreen shape: rounded;";
    elif "Constant" in surfaces[1] or "CornerRounded" in surfaces[1] and "leftCornerRounded: false" in surfaces[1]:
      real_text = real_text + f" windscreen shape: flat;"

    if len(surfaces) == 3:
      boot_size = float(re.search("{ new List<float>\(\){ (.*)f, (.*)f }", surfaces[2]).group(2))
      if boot_size >= 0.9:
        real_text = real_text + " boot size: small;"
      elif boot_size <= 0.8:
        real_text = real_text + " boot size: medium;"
    if (len(surfaces) == 2 and "CornerRounded" in surfaces[1] and "rightCornerRounded: true" in surfaces[1]) or (len(surfaces) == 3 and ("TotalRounded" in surfaces[2] or "CornerRounded" in surfaces[2] and "rightCornerRounded: true" in surfaces[1])):
      real_text = real_text + " boot shape: rounded;"
    elif (len(surfaces) == 2 and ("CornerRounded" in surfaces[1] and "rightCornerRounded: false" in surfaces[1]) or "Constant" in surfaces[1]) or (len(surfaces) == 3 and ("Constant" in surfaces[2] or "CornerRounded" in surfaces[1] and "rightCornerRounded: false" in surfaces[1])):
      real_text = real_text + " boot shape: flat;"
    real_text = real_text + "\n"
    if real_text == input_text:
      return True
    else:
      return False
  except:
    return False

In [27]:
request = "Generate a car with a large bonnet."
system = """You are an expert in writing code to generate car geometry. You receive a description of a car containing its geometric characteristics. Write code in the format of a C# class definition that describes the car geometry. Fill in the class properties according to the user's request."""
prompt = f"<s><<SYS>>{system}<</SYS>>[INST]{request}[/INST]"
tokenized_prompt = tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
gen_config = GenerationConfig.from_pretrained(model.name_or_path)
with torch.inference_mode():
    output = model.generate(**{"input_ids":tokenized_prompt, "generation_config":gen_config})
output = tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)
print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[SHAPE]Give the car a flat front with a raised lip at the edges. The bonnet should be large and rounded at the edges. The sides should be straight and flat. The rear should be flat with a small vertical lip at the top. The bottom should be rounded and raised high off the ground. The wheels should be large and rounded at the edges with thin spokes. [/SHAPE]

<s><<SYS>>You are an expert in writing code to generate car geometry. You receive a description of a car containing its geometric characteristics. Write code in the format of a C# class definition that describes the car geometry. Fill in the class properties according to the user's request.<</SYS>>[INST]Generate a car with a large bonnet.[/INST][SHAPE]Give the car a flat front with a raised lip at the edges. The bonnet should be large and rounded at the edges. The sides should be straight and flat. The rear should be flat with a small vertical lip at the top. The bottom should be rounded and raised high off the ground. The wheels sh